In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")  # langsmith tracking
os.environ["LANGSMITH_TRACING"] = os.getenv("LANGSMITH_TRACING")
os.environ["LANGSMITH_PROJECT"] = os.getenv("LANGSMITH_PROJECT")
os.environ["LANGSMITH_ENDPOINT"] = os.getenv("LANGSMITH_ENDPOINT")

In [3]:
# Data Ingestion (scrape data from webpage)

from langchain_community.document_loaders import WebBaseLoader


In [4]:
loader = WebBaseLoader(
    "https://docs.smith.langchain.com/administration/how_to_guides"
)

In [5]:
loader.load()

[Document(metadata={'source': 'https://docs.smith.langchain.com/administration/how_to_guides', 'title': 'Administration how-to guides | 🦜️🛠️ LangSmith', 'description': 'Step-by-step guides that cover key tasks and operations in LangSmith.', 'language': 'en'}, page_content='\n\n\n\n\nAdministration how-to guides | 🦜️🛠️ LangSmith\n\n\n\n\n\n\nSkip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationTutorialsOptimize tracing spend on LangSmithHow-to GuidesSetupConceptual GuideSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceAdministrationHow-to GuidesOn this pageAdministration how-to guides\nStep-by-step guides that cover key tasks and operations in LangSmith.\nOrganizat

In [9]:
## Chunking
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200
)
documents = splitter.split_documents(loader.load())

In [10]:
## Embedding creation

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [ ]:
## Vector store creation

from langchain_community.vectorstores import FAISS 

db = FAISS.from_documents(
    documents,
    embeddings
)

In [ ]:
## query from vector store

query = "Step-by-step guides that cover key tasks and operations in LangSmith."

result = db.similarity_search(
    query=query
)

In [15]:
result[0].page_content

'Administration how-to guides | 🦜️🛠️ LangSmith\n\n\n\n\n\n\nSkip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationTutorialsOptimize tracing spend on LangSmithHow-to GuidesSetupConceptual GuideSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceAdministrationHow-to GuidesOn this pageAdministration how-to guides\nStep-by-step guides that cover key tasks and operations in LangSmith.\nOrganization Management\u200b\nSee the following guides to set up your LangSmith account.\n\nCreate an account and API key\nSet up an organization\n\nCreate an organization\nManage and navigate workspaces\nManage users\nManage your organization using the API\n\n\nSet up a workspace'

In [18]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# prompt creation
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", """Answer the following questions based only on the provided context:
<context>
{context}
</context>""")
])

# llm setup
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    model="gpt-4o"
)

# retrival chain
document_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt
)

from langchain_core.documents import Document
result = document_chain.invoke(
    {
        "input":query,
        "context": [
            Document(page_content=query)
        ]
    }
)

In [19]:
result

'Certainly! Please go ahead and ask your questions related to the step-by-step guides and tasks in LangSmith.'

In [20]:
## Retriver
retriver = db.as_retriever()

from langchain.chains import create_retrieval_chain 
retriever_chain = create_retrieval_chain(
    retriver,
    document_chain
)

In [22]:
response = retriever_chain.invoke(
    {
        "input":query
    }
)

In [23]:
response

{'input': 'Step-by-step guides that cover key tasks and operations in LangSmith.',
 'context': [Document(id='6c0431e6-428a-4a52-b02e-3fc4ddc02352', metadata={'source': 'https://docs.smith.langchain.com/administration/how_to_guides', 'title': 'Administration how-to guides | 🦜️🛠️ LangSmith', 'description': 'Step-by-step guides that cover key tasks and operations in LangSmith.', 'language': 'en'}, page_content='Administration how-to guides | 🦜️🛠️ LangSmith\n\n\n\n\n\n\nSkip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationTutorialsOptimize tracing spend on LangSmithHow-to GuidesSetupConceptual GuideSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceAdministrationHow-to 

In [25]:
response["answer"]

'Here are answers to questions based on the provided context:\n\n1. **What types of guides are provided in the LangSmith Administration section?**\n   - The LangSmith Administration section provides step-by-step guides for key tasks and operations such as organization management, setting up a workspace, setting up billing, and setting up resource tags.\n\n2. **What tasks are covered under Organization Management?**\n   - Organization Management covers tasks like creating an account and API key, setting up an organization, managing and navigating workspaces, managing users, and managing your organization using the API.\n\n3. **What can you do under Workspace Setup?**\n   - Under Workspace Setup, you can create a workspace, manage users, and configure workspace settings.\n\n4. **What billing-related tasks can be managed?**\n   - You can update the invoice email, tax ID, and business information as well as set up access control for enterprise accounts.\n\n5. **What can you do with resourc